## Dataset: https://www.kaggle.com/datasets/saurabhbadole/latest-data-science-job-salaries-2024/

In [1]:
import pandas as pd

p = pd.read_csv('../data/jobs_in_data_2020_2024.csv')

## Data Exploration

In [2]:
p

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2021,MI,FT,Data Scientist,30400000,CLP,40038,CL,100,CL,L
1,2021,MI,FT,BI Data Analyst,11000000,HUF,36259,HU,50,US,L
2,2020,MI,FT,Data Scientist,11000000,HUF,35735,HU,50,HU,L
3,2021,MI,FT,ML Engineer,8500000,JPY,77364,JP,50,JP,S
4,2022,SE,FT,Lead Machine Learning Engineer,7500000,INR,95386,IN,50,IN,L
...,...,...,...,...,...,...,...,...,...,...,...
14833,2022,MI,FT,Business Intelligence Developer,15000,USD,15000,GH,100,GH,M
14834,2020,EX,FT,Staff Data Analyst,15000,USD,15000,NG,0,CA,M
14835,2021,EN,FT,Machine Learning Developer,15000,USD,15000,TH,100,TH,L
14836,2022,EN,FT,Data Analyst,15000,USD,15000,ID,0,ID,L


In [3]:
p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14838 entries, 0 to 14837
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           14838 non-null  int64 
 1   experience_level    14838 non-null  object
 2   employment_type     14838 non-null  object
 3   job_title           14838 non-null  object
 4   salary              14838 non-null  int64 
 5   salary_currency     14838 non-null  object
 6   salary_in_usd       14838 non-null  int64 
 7   employee_residence  14838 non-null  object
 8   remote_ratio        14838 non-null  int64 
 9   company_location    14838 non-null  object
 10  company_size        14838 non-null  object
dtypes: int64(4), object(7)
memory usage: 1.2+ MB


In [4]:
p.nunique()

work_year                5
experience_level         4
employment_type          4
job_title              153
salary                2363
salary_currency         23
salary_in_usd         2730
employee_residence      88
remote_ratio             3
company_location        77
company_size             3
dtype: int64

In [5]:
p.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [6]:
p.describe()

,work_year,salary,salary_in_usd,remote_ratio
count,14838.000000,1.483800e+04,14838.000000,14838.000000
mean,2023.138900,1.650227e+05,149874.718763,32.760480
std,0.700799,3.562354e+05,69009.181349,46.488278
min,2020.000000,1.400000e+04,15000.000000,0.000000
25%,2023.000000,1.021000e+05,102000.000000,0.000000
50%,2023.000000,1.422000e+05,141300.000000,0.000000
75%,2024.000000,1.875000e+05,185900.000000,100.000000
max,2024.000000,3.040000e+07,800000.000000,100.000000


In [7]:
# - https://www.kaggle.com/code/murilozangari/jobs-data-field-2024-eda-salary-estimation

# Check for duplicates
duplicate_rows = p[p.duplicated()]

# Print the number of duplicates
num_duplicates = duplicate_rows.shape[0]
print(f"Number of duplicate rows:", num_duplicates, "\nPercentege from the total:", round(num_duplicates/len(p),3)*100)

Number of duplicate rows: 5711 
Percentege from the total: 38.5


# Preprocessing

In [9]:
p['work_year'] = p['work_year'].astype(str)
p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14838 entries, 0 to 14837
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           14838 non-null  object
 1   experience_level    14838 non-null  object
 2   employment_type     14838 non-null  object
 3   job_title           14838 non-null  object
 4   salary              14838 non-null  int64 
 5   salary_currency     14838 non-null  object
 6   salary_in_usd       14838 non-null  int64 
 7   employee_residence  14838 non-null  object
 8   remote_ratio        14838 non-null  int64 
 9   company_location    14838 non-null  object
 10  company_size        14838 non-null  object
dtypes: int64(3), object(8)
memory usage: 1.2+ MB


In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Separate the target variable from the predictors
X = p.drop('salary', axis=1)  
X = p.drop('salary_in_usd', axis=1)  
# Set target variable
y = p['salary_in_usd'] 

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Identify numerical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create preprocessor
preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_cols), ('passthrough', 'passthrough', numerical_cols)])

# Create the XGBoost regressor
model = XGBRegressor()

# Create the pipeline with preprocessor and model
regressor = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# XGBoost using GA and the pyeasyga framework

In [11]:
from deap import creator, base, tools, algorithms
import random
import time

# Define the fitness function and individual
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Define the toolbox
toolbox = base.Toolbox()

# Small grid
# Define the attributes for individuals
toolbox.register("attr_min_child_weight", random.choice, [1, 5, 10])
toolbox.register("attr_gamma", random.choice, [0, 0.2, 0.5, 1, 1.5, 2, 5])
toolbox.register("attr_subsample", random.choice, [0.6, 0.8, 1.0])
toolbox.register("attr_colsample_bytree", random.choice, [0.6, 0.8, 1.0])
toolbox.register("attr_max_depth", random.choice, [3, 4, 5, 6])
toolbox.register("attr_learning_rate", random.choice, [0.01, 0.05, 0.1])
toolbox.register("attr_n_estimators", random.choice, [100, 200, 300, 400, 500])

# Register the individual
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_n_estimators, toolbox.attr_max_depth, toolbox.attr_learning_rate,
                  toolbox.attr_colsample_bytree, toolbox.attr_gamma, toolbox.attr_subsample, toolbox.attr_min_child_weight), n=1)

# Register the population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define the evaluation function
def evaluate(individual):
    # Set parameters
    params = {
        'model__n_estimators': individual[0],           # n_estimators from the individual
        'model__max_depth': individual[1],              # max_depth from the individual
        'model__learning_rate': individual[2],          # learning_rate from the individual
        'model__colsample_bytree': individual[3],       # colsample_bytree from the individual
        'model__gamma': individual[4],                  # gamma from the individual
        'model__subsample': individual[5],              # subsample from the individual
        'model__min_child_weight': individual[6]        # min_child_weight from the individual
    }
    regressor.set_params(**params)

    # Run evaluation
    scores = cross_validate(regressor, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
    return np.mean(scores['test_score']),

# Register evaluation function
toolbox.register("evaluate", evaluate)

# -------------------- Configure GA parameters --------------------
pop = 10
gen = 40
local_mutp = 0.2
global_mutp = 0.2
crossop = 0.9
# -----------------------------------------------------------------

# Mutation function for integer and float attributes with custom mutation probability
def mutate_individual(individual, mutp):
    if random.random() < mutp:
        individual[0] = random.randint(100, 500)
    
    if random.random() < mutp:
        individual[1] = random.randint(3, 6)
    
    if random.random() < mutp:
        individual[2] = random.uniform(0.01, 0.1)
    
    if random.random() < mutp:
        individual[3] = random.uniform(0.6, 1.0)
    
    if random.random() < mutp:
        individual[4] = random.uniform(0, 5)
    
    if random.random() < mutp:
        individual[5] = random.uniform(0.6, 1.0)
    
    if random.random() < mutp:
        individual[6] = random.randint(1, 10)

    return individual,

# Define the genetic operators
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate_individual, mutp=local_mutp)
toolbox.register("select", tools.selTournament, tournsize=3)

# Initialize the population and hall of fame
pop = toolbox.population(n=pop)
hof = tools.HallOfFame(1)

# Run the genetic algorithm
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

start_time = time.time()
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=crossop, mutpb=global_mutp, ngen=gen, stats=stats, halloffame=hof, verbose=True)
end_time = time.time()

gen	nevals	avg     	min     	max     
0  	10    	0.960178	0.835031	0.984024
1  	8     	0.972039	0.90432 	0.984024
2  	10    	0.983357	0.979275	0.984143
3  	10    	0.98375 	0.982423	0.984089
4  	10    	0.984031	0.983601	0.984186
5  	10    	0.984254	0.984024	0.984669
6  	10    	0.984344	0.984089	0.984798
7  	7     	0.98387 	0.980715	0.984798
8  	9     	0.984567	0.984182	0.984798
9  	10    	0.984703	0.984441	0.984798
10 	7     	0.983904	0.978554	0.984798
11 	6     	0.984459	0.981411	0.984798
12 	10    	0.984798	0.984798	0.984798
13 	8     	0.984798	0.984798	0.984798
14 	8     	0.984805	0.984798	0.984869
15 	9     	0.984409	0.980696	0.984869
16 	10    	0.984487	0.981264	0.984869
17 	8     	0.984749	0.983672	0.984869
18 	10    	0.984869	0.984869	0.984869
19 	9     	0.984006	0.980307	0.984978
20 	8     	0.984033	0.976078	0.984978
21 	8     	0.984223	0.98015 	0.984978
22 	8     	0.984978	0.984978	0.984978
23 	10    	0.98477 	0.9829  	0.984978
24 	10    	0.984278	0.980749	0.984978
25 	10    	0

In [12]:
# Retrieve best individual and parameters
best_individual = hof[0]
best_params = { 
    'model__n_estimators': best_individual[0],           
    'model__max_depth': best_individual[1],             
    'model__learning_rate': best_individual[2],        
    'model__colsample_bytree': best_individual[3],    
    'model__gamma': best_individual[4],                 
    'model__subsample': best_individual[5],            
    'model__min_child_weight': best_individual[6]      
}

print("Best parameters: ", best_params)

regressor.set_params(**best_params)
# Fit the model
regressor.fit(X_train, y_train)

Best parameters:  {'model__n_estimators': 480, 'model__max_depth': 3, 'model__learning_rate': 0.049007052687965555, 'model__colsample_bytree': 0.970034704433829, 'model__gamma': 4.880194587984013, 'model__subsample': 0.9698017878293229, 'model__min_child_weight': 1}


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['work_year',
                                                   'experience_level',
                                                   'employment_type',
                                                   'job_title',
                                                   'salary_currency',
                                                   'employee_residence',
                                                   'company_location',
                                                   'company_size']),
                                                 ('pass...
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.049007052687965555, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=3, max_leaves=None,
                              min_child_weight=1, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=480, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [13]:
# Validate the model
cv_results = cross_validate(regressor, X_train, y_train, cv=5, scoring='r2')
avg_cv_result = np.mean(np.abs(cv_results['test_score']))

print(abs(cv_results['test_score']))
print(avg_cv_result)

[0.99381493 0.98371613 0.9860645  0.97453147 0.98802089]
0.9852295832667373


In [14]:
# Calculate the test metrics
test_predictions = regressor.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
test_r2 = r2_score(y_test, test_predictions)

print("Test RMSE:", test_rmse)
print("Test NRMSE score:", test_rmse / (y.max() - y.min()))
print("Test R2 Score:", test_r2)
print(f"Tuning completed in: {end_time-start_time:.6f} seconds.")

Test RMSE: 11158.545966183017
Test NRMSE score: 0.014214708237175818
Test R2 Score: 0.9749313398226781
Tuning completed in: 160.339815 seconds.


In [30]:
import plotly.graph_objects as go

# Extract the fitness values from the log
max_fitness_values = [gen['max'] for gen in log]
avg_fitness_values = [gen['avg'] for gen in log]

# Initialize the plot
fig = go.Figure()

# Add line for max fitness values
fig.add_trace(go.Scatter(x=list(range(len(max_fitness_values))), y=max_fitness_values, 
                         mode='lines+markers', name='Max Fitness'))

# Add line for average fitness values
fig.add_trace(go.Scatter(x=list(range(len(avg_fitness_values))), y=avg_fitness_values, 
                         mode='lines+markers', name='Average Fitness'))

# Update layout
fig.update_layout(
    title='Fitness history per generation',
    xaxis_title='Generation',
    yaxis_title='Fitness',
    autosize=False,
    width=900,  # Adjust the width of the figure
    height=400  # Adjust the height of the figure
)


fig.show()

In [19]:
# import plotly.io as py

# py.write_html(fig, 'fitness_history.html')